## exercicio 1

In [93]:
import numpy as np
conec = np.array([[1,2],[2,3],[3,4],[4,5],[5,2],[5,3],[5,1]])
C= np.array ([2,2,1,2,1,2,2])

In [94]:
#

def Assembly(conec,C):
    nv = max(conec[:,0])#numero de nos 
    nc = len(conec[:,0])# numero de linhas

    A = np.zeros((nv,nv))
    for i in range(nc):
        n1=conec [i,0]-1
        n2=conec [i,1]-1

        A[n1,n2] += -C[i]
        A[n2,n1] += -C[i]
        A[n1,n1] += C[i]
        A[n2,n2] += C[i]

    return A


In [95]:
print('A = \n',Assembly(conec,C))

A = 
 [[ 4. -2.  0.  0. -2.]
 [-2.  5. -2.  0. -1.]
 [ 0. -2.  5. -1. -2.]
 [ 0.  0. -1.  3. -2.]
 [-2. -1. -2. -2.  7.]]


## exercicios 2

In [96]:
def SolveNetwork(conec, C, natm, nB, QB):
    Atilde = Assembly (conec, C)
    natm -= 1
    nB -= 1
    Atilde [natm,:] = 0
    Atilde [natm,natm] = 1

    b = np.zeros((len(Atilde),1))
    b[nB] = QB
    pressure = np.linalg.solve(Atilde,b)

    return pressure
    

In [97]:
P = SolveNetwork(conec, C, 3, 1, 3)
print(np.round(P, 3))

[[1.398]
 [0.682]
 [0.   ]
 [0.409]
 [0.614]]


## exercicio 03


In [98]:
def KD(conec, C):
    nv = max(conec[:, 0])               # maximo de nos
    nc = len(conec[:, 0])               # numero de linhas
    K = np.zeros(shape=(nc,nc))
    D = np.zeros(shape=(nc,nv))
    
    for i in range(nc):     #diag principal
        K[i,i] = C[i]

    for k in range(nc):                  
        n1 = conec[k,0]-1
        n2 = conec[k,1]-1


        D[k,n1] = 1
        D[k,n2] = -1 

    return K, D

In [99]:
def Q(conec, C, P):
    nv = max(conec[:, 0])       # numero de nos
    nc = len(conec[:, 0])       # numero de canos
    
           
    K = np.zeros(shape=(nc,nc))
    D = np.zeros(shape=(nc,nv))

    K, D = KD(conec, C)

    return ((K @ D) @ P)

In [100]:
K, D = KD(conec, C)
print(">> K: \n", K, '\n')
print(">> D: \n", D)

>> K: 
 [[2. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 2.]] 

>> D: 
 [[ 1. -1.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.]
 [ 0.  0.  1. -1.  0.]
 [ 0.  0.  0.  1. -1.]
 [ 0. -1.  0.  0.  1.]
 [ 0.  0. -1.  0.  1.]
 [-1.  0.  0.  0.  1.]]


### Damn it 

## Exercicio 04

In [101]:
def calcula_potencia(conec,P):
    nv = max(conec[:, 0])              # numero de nos
    nc = len(conec[:, 0])       # numero de canos
    
    K = np.zeros(shape=(nc,nc))
    D = np.zeros(shape=(nc,nv))

    K, D = KD(conec, C)
    return (np.transpose(P)@(np.transpose(D) @ K @ D ) @ P )


## Exercicio 

In [102]:
def Assembly2(conec, C, nv, nc):
    A = np.zeros(shape=(nv,nv))
    for k in range(nc):
        n1 = conec[k,0]
        n2 = conec[k,1]

        A[n1,n2] += -C[k]
        A[n2,n1] += -C[k]
        A[n1,n1] += C[k] 
        A[n2,n2] += C[k]
    
    return A

def SolveNetwork(conec, C, natm, nB, QB, nv, nc):
    Atilde = Assembly2(conec, C, nv, nc)
    natm -= 1
    nB -= 1
    Atilde[natm, :] = 0
    Atilde[natm, natm] = 1
    
    b = np.zeros((len(Atilde),1))
    b[nB] = QB
    pressure = np.linalg.solve(Atilde, b)
    return pressure

In [103]:
def GeraRede(n,m,CH,CV):
    nv = n*m
    nc = (n-1)*m + (m-1)*n
    
    coord = np.zeros(shape=(nv,2))

    for i in range(n):
      for j in range(m):
        ig = i + j*n
        coord[ig,0] = i
        coord[ig,1] = j

    conec = np.zeros(shape=(nc,2), dtype=int)
    C = np.zeros(nc)
    
    # Loop sobre canos horizontais
    for j in range(m):
        for i in range(n-1):
          k = j*(n-1) + i
          conec[k,0] = j*n + i
          conec[k,1] = j*n + i+1
          C[k] = CH

    # Loop sobre canos verticais
    for i in range(n):
      for j in range(m-1):
          k = (n-1)*m + j*n + i
          conec[k,0] = i + j*n
          conec[k,1] = i + (j+1)*n
          C[k] = CV

    return nv, nc, conec, C, coord
    
def PlotPressure(m, n, pressure):
    x = np.arange(0, m, 1)
    y = np.arange(0, n, 1)
    X,Y=np.meshgrid(x,y)
    Z = np.copy(pressure)
    Z.shape = (n,m)
    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_aspect('equal')
    ax.set(xlabel='x', ylabel='y', title='Contours of pressure')
    ax.grid()
    ax.set_xticks(x)
    ax.set_yticks(y)
    cp = plt.contourf(X, Y, Z, cmap='jet')
    cb = plt.colorbar(cp)
    plt.show()

def PlotPressureAsSurface(m, n, pressure):
    x = np.arange(0, m, 1)
    y = np.arange(0, n, 1)
    X,Y=np.meshgrid(x,y)
    Z = np.copy(pressure)
    Z.shape = (n,m)
    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_aspect('equal')
    ax.set(xlabel='x', ylabel='y', title='Contours of pressure')
    ax.grid()
    ax.set_xticks(x)
    ax.set_yticks(y)
    ax = plt.axes(projection="3d")
    ax.plot_surface(X, Y, Z, cmap='jet', edgecolor=None)
    ax.set(xlabel='$x$', ylabel='$y$', zlabel='$z$')
    ax.view_init(elev=30., azim=45)
    plt.show()

In [104]:
m = 10
n = 12

nv2, nc2, conec2, C2, coord = GeraRede(m,n,1.3, 3.6)
#print(nv, '\n', nc, '\n', conec, '\n', C, '\n', coord)

In [105]:


P = SolveNetwork(conec2, C2, 120, 1, 3, nv2, nc2)
PlotPressure(m,n, P)
PlotPressureAsSurface(10,12, P)

NameError: name 'plt' is not defined